In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace,WebSearchTool, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import os
from pydantic import BaseModel
from agents.model_settings import ModelSettings

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI


In [4]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

### SYSTEM TOOLS

In [5]:
from instructions import websearch_instruction, app_launcher_instruction, file_searcher_instruction, clipboard_manager_instruction

In [6]:
from launch_app import launch_app

In [7]:
APP_LAUNCHER_AGENT=Agent(name="Gemini App laucher", instructions=app_launcher_instruction, model=gemini_model, tools=[launch_app])

In [8]:
# with trace("App launcher"):
#     result = await Runner.run(APP_LAUNCHER_AGENT, "please open chrome")

In [9]:
# print(result.final_output)

In [10]:
from search_file import search_file

In [11]:
FILE_SEARCHER_AGENT=Agent(name="Gemini Folder/File searcher", instructions=file_searcher_instruction, model="gpt-4o-mini", tools=[search_file])

In [12]:
# with trace("File searcher"):
#     result = await Runner.run(FILE_SEARCHER_AGENT, "Can you please tell me where is pdf_data.json file")

In [13]:
# print(result.final_output)

In [14]:
from clipboard_manager import save_clipboard, get_clipboard_history, get_last_clipboard

In [15]:
CLIPBOARD_MANAGER=Agent(name="Gemini  clipboard manager", instructions=clipboard_manager_instruction, model=gemini_model, tools=[save_clipboard, get_clipboard_history, get_last_clipboard])

In [16]:
# with trace("clipboard manager"):
#     result = await Runner.run(CLIPBOARD_MANAGER, "can you please save my clip board history, and show it to me")

### INTERNET TOOLS

In [17]:
from wikipedia_tool import search_wikipedia

In [18]:

SEARCH_AGENT = Agent(
    name="Search agent",
    instructions=websearch_instruction,
    tools=[WebSearchTool(search_context_size="low"),search_wikipedia],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [19]:
# message = "What is AI and how it works"

# with trace("Search"):
#     result = await Runner.run(SEARCH_AGENT, message)
# print(result.final_output)

In [20]:
from docker_runner import run_code_in_docker
from instructions import code_helper_instruction

In [21]:
print(code_helper_instruction)

You are an expert coding assistant with deep knowledge of multiple programming languages, frameworks, and tools. Your task is to generate correct, optimized, and well-documented code in a single attempt based on the user’s request. You are also provided with a tool (Docker) to test your code safely if it is Python or Node.js. Always use print statements (or console.log in JavaScript) to output final results, otherwise the execution will not show anything. Guidelines: 1. Understanding & Accuracy - Read the request carefully before answering. Choose the most suitable programming language, framework, or library unless the user specifies one. Always ensure code is executable without errors. Do not include hidden reasoning or explanations unless asked. 2. Code Quality - Use clean, maintainable, and well-structured code. Apply best practices (naming conventions, modularity, reusability). Handle edge cases where applicable. Include necessary imports, environment setup, or configuration if req

In [22]:

CODING_AGENT = Agent(
    name="coding agent",
    instructions=code_helper_instruction,
    tools=[run_code_in_docker],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [23]:
message = 'Write a python program to calculate the first 10,000 terms \
    of this series, multiplying the total by 4: 1 - 1/3 + 1/5 - 1/7 + ...'

with trace("CODING_AGENT"):
    result = await Runner.run(CODING_AGENT, message)
print(result.final_output)

Here’s the Python program that calculates the first 10,000 terms of the series \( 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \ldots \) and multiplies the total by 4:

```python
# Calculate the first 10,000 terms of the series: 1 - 1/3 + 1/5 - 1/7 + ...

def calculate_series(n_terms):
    total = 0
    for i in range(n_terms):
        term = (-1) ** i / (2 * i + 1)  # Compute each term
        total += term  # Add or subtract based on the term's sign
    return total * 4  # Multiply the final total by 4

# Number of terms to compute
num_terms = 10000
result = calculate_series(num_terms)
print(result)  # Output the result
```

### Output
The result of calculating the series is approximately \( 3.1414926535900345 \). 

This value is an approximation of \( \pi \), demonstrating how the series converges to this mathematical constant.
